In [15]:
!pip3 install -U -q sagemaker boto3 langchain streamlit ipykernel pypdf "ai21[AWS]" faiss-cpu

In [9]:
import json
from typing import List
import time 
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import boto3

from load_embeddings import create_sagemaker_embeddings_from_js_model, encoder_name, encoder_endpoint_name



In [11]:
boto3_session = boto3.session.Session()
print(boto3_session.region_name)

us-west-2


In [16]:
loader = PyPDFDirectoryLoader('docs/')
data = loader.load()
## split the documents in 1024 characters and consequent documents 
## will have 60 character overlap 
### these parmaters should be tuned 
CHUNK_SIZE=1024
CHUNK_OVERLAP=64
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, 
                                               chunk_overlap=CHUNK_OVERLAP,
                                               length_function = len,
                                               add_start_index = True,)
docs = text_splitter.split_documents(data)
embeddings = create_sagemaker_embeddings_from_js_model(encoder_endpoint_name, 
                                                       boto3_session.region_name)

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    text_splitter=text_splitter
)
docsearch = FAISS.from_documents(docs, embeddings)

In [17]:
question="Tell me about the return of office policy"
results = docsearch.similarity_search(question, k=3)
print(results[0].page_content)
print(results[0].metadata)

We also looked hard at how we were working together as a team and asked our corporate employees to come
back to the office at least three days a week , beginning in May. During the pandemic, our employees rallied to
{'source': 'docs/2022-Shareholder-Letter.pdf', 'page': 1, 'start_index': 1111}


In [18]:
question="how are the macroeconomic conditions"
results = docsearch.similarity_search(question, k=3)
for result in results:
    print(result.page_content)

There have also been times when macroeconomic conditions or operating inefficiencies have presented us
commitment and effort from our employees all over the world. I’m not sure any of us would have gotten
work to do, but we’re pleased with our trajectory and the meaningful upside in front of us.


In [19]:
docsearch.save_local("shareholders_letter.faiss")